Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Extractive Summarization on CNN/DM Dataset using Transformer Version of BertSum


### Summary

This notebook demonstrates how to fine tune Transformers for extractive text summarization. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring, result postprocessing, and model evaluation.

BertSum refers to  [Fine-tune BERT for Extractive Summarization](https://arxiv.org/pdf/1903.10318.pdf) with [published example](https://github.com/nlpyang/BertSum/). And the Transformer version of Bertsum refers to our modification of BertSum and the source code can be accessed at (https://github.com/daden-ms/BertSum/). 

Extractive summarization are usually used in document summarization where each input document consists of mutiple sentences. The preprocessing of the input training data involves assigning label 0 or 1 to the document sentences based on the give summary. The summarization problem is also simplfied to classifying whether a document sentence should be included in the summary. 

The figure below illustrates how BERTSum can be fine tuned for extractive summarization task. [CLS] token is inserted at the beginning of each sentence, so is [SEP] token at the end. Interval segment embedding and positional embedding are added upon the token embedding as the input of the BERT model. The [CLS] token representation is used as sentence embedding and only the [CLS] tokens are used as the input for the summarization model. The summarization layer predicts the probability for each  sentence being included in the summary. Techniques like trigram blocking can be used to improve model accuarcy.   

<img src="https://nlpbp.blob.core.windows.net/images/BertSum.PNG">


### Before You Start

The running time shown in this notebook is on a Standard_NC24s_v3 Azure Ubuntu Virtual Machine with 4 NVIDIA Tesla V100 GPUs. 
> **Tip**: If you want to run through the notebook quickly, you can set the **`QUICK_RUN`** flag in the cell below to **`True`** to run the notebook on a small subset of the data and a smaller number of epochs. 

Using only 1 NVIDIA Tesla V100 GPUs, 16GB GPU memory configuration,
- for data preprocessing, it takes around 1 minutes to preprocess the data for quick run. Otherwise it takes ~20 minutes to finish the data preprocessing. This time estimation assumes that the chosen transformer model is "distilbert-base-uncased" and the sentence selection method is "greedy", which is the default. The preprocessing time can be significantly longer if the sentence selection method is "combination", which can achieve better model performance.

- for model fine tuning, it takes around 2 minutes for quick run. Otherwise, it takes around ~3 hours to finish. This estimation assumes the chosen encoder method is "transformer". The model fine tuning time can be shorter if other encoder method is chosen, which may result in worse model performance. 

### Additional Notes

* **ROUGE Evalation**: To run rouge evaluation, please refer to the section of compute_rouge_perl in [summarization_evaluation.ipynb](./summarization_evaluation.ipynb) for setup.

* **Distributed Training**:
Please note that the jupyter notebook only allows to use pytorch [DataParallel](https://pytorch.org/docs/master/nn.html#dataparallel). Faster speed and larger batch size can be achieved with pytorch [DistributedDataParallel](https://pytorch.org/docs/master/notes/ddp.html)(DDP). Script [extractive_summarization_cnndm_distributed_train.py](./extractive_summarization_cnndm_distributed_train.py) shows an example of how to use DDP.



In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
## Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs.
QUICK_RUN = True
## Set USE_PREPROCSSED_DATA = True to skip the data preprocessing
USE_PREPROCSSED_DATA = True

### Configuration


In [3]:
import os
import shutil
import sys
from tempfile import TemporaryDirectory
import torch

nlp_path = os.path.abspath("../../")
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

# from utils_nlp.dataset.cnndm import CNNDMBertSumProcessedData, CNNDMSummarizationDataset
# from utils_nlp.dataset.bundesministerium import BUNDBertSumProcessedData, BUNDSummarizationDataset
from utils_nlp.dataset.swiss import SwissSummarizationDataset

from utils_nlp.eval import compute_rouge_python, compute_rouge_perl
from utils_nlp.models.transformers.extractive_summarization import (
    ExtractiveSummarizer,
    ExtSumProcessedData,
    ExtSumProcessor,
)

from utils_nlp.models.transformers.datasets import SummarizationDataset
import nltk
from nltk import tokenize

import pandas as pd
import scrapbook as sb
import pprint

/home/ubuntu/anaconda3/envs/nlp_cpu/lib/python3.6/site-packages/dask/dataframe/utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
# ARTICLE_TO_SUMMARIZE = """
# Italien erlaubt nach tagelangem Zögern den etwa 180 Migranten auf dem privaten Rettungsschiff "Ocean Viking" den Wechsel auf das italienische Quarantäne-Schiff "Moby Zaza". Die Übernahme der aus Seenot geretteten Menschen sei für Montag geplant, hieß es am Samstagabend aus Quellen im Innenministerium in Rom. Zuvor hatte sich die Lage auf dem Schiff der Organisation SOS Méditerranée, das sich in internationalen Gewässern vor Sizilien befindet, zugespitzt.
# Die Betreiber berichteten demnach von einem Hungerstreik unter den Geflüchteten. Verena Papke, Geschäftsführerin von SOS Méditerranée für Deutschland, hatte am Freitag von mehreren Suizidversuchen gesprochen. Die "Ocean Viking" hatte zudem den Notstand an Bord ausgerufen. Bis dahin waren mehrere Bitten um Zuweisung eines sicheren Hafens in Malta und Italien erfolglos geblieben.

# Corona-Abstriche bei den Migranten geplant
# Die Crew sandte die dringende Anfrage an die Behörden beider Länder zur Aufnahme von rund 45 Menschen, die in schlechter Verfassung seien. Italien schickte daraufhin am Samstag einen Psychiater und einen kulturellen Mediator aus Pozzallo für mehrere Stunden an Bord, berichteten beide Seiten. Danach kam die Erlaubnis aus Rom zur Übernahme auf die "Moby Zaza". Die Lage an Bord habe sich jedoch etwas entspannt, hieß es aus der italienischen Hauptstadt. Am Sonntag seien zunächst Corona-Abstriche bei den Migranten geplant.

# Wie SOS Méditerranée am Samstag schrieb, nahm das Schiff in insgesamt vier Einsätzen am 25. und am 30. Juni etwa 180 Menschen aus dem Mittelmeer an Bord. Italien und Malta hatten sich in der Corona-Pandemie zu nicht sicheren Häfen erklärt. Trotzdem brechen Migranten von Libyen und Tunesien in Richtung Europa auf. Rom und Valletta nahmen zuletzt zwar wieder Menschen von privaten Schiffen auf, doch die Länder zögern mit der Zuweisung von Häfen oft lange. Sie fordern von anderen EU-Staaten regelmäßig Zusagen über die Weiterverteilung der Menschen."""

In [5]:
# from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

# # see ``examples/summarization/bart/evaluate_cnn.py`` for a longer example
# model = BartForConditionalGeneration.from_pretrained('airKlizz/bart-large-multi-de-wiki-news')
# tokenizer = BartTokenizer.from_pretrained('airKlizz/bart-large-multi-de-wiki-news')
# inputs = tokenizer.batch_encode_plus([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
# # Generate Summary
# summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=30, max_length=1000, early_stopping=True)
# print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

In [6]:
# from transformers import BartTokenizer, BartModel

# # tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# # model = BartModel.from_pretrained('facebook/bart-large')

# inputs = tokenizer.encode_plus("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

# last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tup


### Configuration: choose the transformer model to be used


Several pretrained models have been made available by [Hugging Face](https://github.com/huggingface/transformers). For extractive summarization, the following pretrained models are supported. 

In [7]:
pd.DataFrame({"model_name": ExtractiveSummarizer.list_supported_models()})

,model_name
0,bert-base-uncased
1,bert-base-german-cased
2,distilbert-base-uncased
3,dbmdz/bert-base-german-uncased
4,bert-base-german-dbmdz-cased
5,bert-base-multilingual-cased
6,distilbert-base-german-cased
7,bert-base-german-dbmdz-uncased
8,severinsimmler/bert-adapted-german-press
9,xlm-roberta-large-finetuned-conll03-german


In [8]:
# Transformer model being used
MODEL_NAME = "distilbert-base-uncased"

# Data being used
DATA_NAME = "swiss"

In [9]:
# notebook parameters
# the cache data path during find tuning
CACHE_DIR = TemporaryDirectory().name
processor = ExtSumProcessor(model_name=MODEL_NAME, cache_dir=CACHE_DIR)

### Data Preprocessing

The dataset we used for this notebook is CNN/DM dataset which contains the documents and accompanying questions from the news articles of CNN and Daily mail. The highlights in each article are used as summary. The dataset consits of ~289K training examples, ~11K valiation examples and ~11K test examples.  You can choose the [Option 1] below preprocess the data or [Option 2] to use the preprocessed version at [BERTSum published example](https://github.com/nlpyang/BertSum/). You don't need to manually download any of these two data sets as the code below will handle downloading. Functions defined specific in [cnndm.py](../../utils_nlp/dataset/cnndm.py) are unique to CNN/DM dataset that's preprocessed by harvardnlp. However, it provides a skeleton of how to preprocessing text into the format that model preprocessor takes: sentence tokenization and work tokenization. 

##### Details of Data Preprocessing

The purpose of preprocessing is to process the input articles to the format that model finetuning needed. Assuming you have (1) all articles and (2) target summaries, each in a file and line-breaker separated, the steps to preprocess the data are:
1. sentence tokenization
2. word tokenization
3. **label** the sentences in the article with 1 meaning the sentence is selected and 0 meaning the sentence is not selected. The algorithms for the sentence selection are "greedy" and "combination" and can be found in [sentence_selection.py](../../utils_nlp/dataset/sentence_selection.py)
3. convert each example to  the desired format for extractive summarization
    - filter the sentences in the example based on the min_src_ntokens argument. If the lefted total sentence number is less than min_nsents, the example is discarded.
    - truncate the sentences in the example if the length is greater than max_src_ntokens
    - truncate the sentences in the example and the labels if the total number of sentences is greater than max_nsents
    - [CLS] and [SEP] are inserted before and after each sentence
    - wordPiece tokenization or Byte Pair Encoding (BPE) subword tokenization
    - truncate the example to 512 tokens
    - convert the tokens into token indices corresponding to the transformer tokenizer's vocabulary.
    - segment ids are generated and added
    - [CLS] token positions are logged
    - [CLS] token labels are truncated if it's greater than 512, which is the maximum input length that can be taken by the transformer model.
    
    
Note that the original BERTSum paper use Stanford CoreNLP for data preprocessing, here we use NLTK for data preprocessing. 

In [10]:
# the data path used to save the downloaded data file
DATA_PATH = '/home/ubuntu/mnt/data/swiss_dataset'
# The number of lines at the head of data file used for preprocessing. -1 means all the lines.
TOP_N = 500
if not QUICK_RUN:
    TOP_N = -1

##### [Option 1] Preprocess  data (Please skip this part if you choose to use preprocessed data)
The code in following cell will download the CNN/DM dataset listed at https://github.com/harvardnlp/sent-summary/.

In [11]:
validation = False

if DATA_NAME is "cnndm":
    train_dataset, test_dataset = CNNDMSummarizationDataset(top_n=TOP_N, local_cache_path=DATA_PATH)
elif DATA_NAME is "swiss":
    if validation:
        train_dataset, validation_dataset, test_dataset = SwissSummarizationDataset(top_n=TOP_N, validation=True, language='german')
    else:
        train_dataset, test_dataset = SwissSummarizationDataset(top_n=TOP_N, validation=False, language='german')
        

In [12]:

len(train_dataset), len(test_dataset)

(475, 25)

### Preprocess the data.

In [13]:

ext_sum_train = processor.preprocess(train_dataset, oracle_mode="greedy")
ext_sum_test = processor.preprocess(test_dataset, oracle_mode="greedy")


### Save the data.

In [14]:
SAVE_DATA = False


# save and load preprocessed data

if SAVE_DATA:
    save_path = os.path.join(DATA_PATH, DATA_NAME + "_processed")
    os.makedirs(save_path, exist_ok=True)

    torch.save(ext_sum_train, os.path.join(save_path, "train_full.pt"))
    torch.save(ext_sum_test, os.path.join(save_path, "test_full.pt"))

In [15]:
len(ext_sum_train)

475

#### Inspect Data

##### [Option 2] Reuse cached preprocessed data

In [18]:
if USE_PREPROCSSED_DATA:
    save_path = os.path.join(DATA_PATH)
    ext_sum_train = torch.load(os.path.join(save_path, "train_full.pt"))
    ext_sum_test = torch.load(os.path.join(save_path, "test_full.pt"))
    

### Model training
To start model training, we need to create a instance of ExtractiveSummarizer.

Potentionally, roberta-based model and xlnet can be supported but needs to be tested.
#### Choose the encoder algorithm.
There are four options:
- baseline: it used a smaller transformer model to replace the bert model and with transformer summarization layer
- classifier: it uses pretrained BERT and fine-tune BERT with **simple logistic classification** summarization layer
- transformer: it uses pretrained BERT and fine-tune BERT with **transformer** summarization layer
- RNN: it uses pretrained BERT and fine-tune BERT with **LSTM** summarization layer

In [19]:
BATCH_SIZE = 5 # batch size, unit is the number of samples
MAX_POS_LENGTH = 512


# GPU used for training
NUM_GPUS = torch.cuda.device_count()

# Encoder name. Options are: 1. baseline, classifier, transformer, rnn.
ENCODER = "transformer"

# Learning rate
LEARNING_RATE=2e-3

# How often the statistics reports show up in training, unit is step.
REPORT_EVERY=50

# total number of steps for training
MAX_STEPS=1e2
# number of steps for warm up
WARMUP_STEPS=5e2
    
if not QUICK_RUN:
    MAX_STEPS=5e4
    WARMUP_STEPS=5e3
 

In [20]:
summarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)

In [21]:
summarizer.model_name

'distilbert-base-uncased'

In [22]:
#"""

summarizer.fit(
            ext_sum_train,
            num_gpus=NUM_GPUS,
            batch_size=BATCH_SIZE,
            gradient_accumulation_steps=2,
            max_steps=MAX_STEPS,
            learning_rate=LEARNING_RATE,
            warmup_steps=WARMUP_STEPS,
            verbose=True,
            report_every=REPORT_EVERY,
            clip_grad_norm=False,
            use_preprocessed_data=False
        )

#"""


Iteration:   0%|          | 0/17100 [00:00<?, ?it/s]

before the while
step:  0
batch:  <utils_nlp.models.transformers.bertsum.data_loader.Batch object at 0x7f8535cccb00>
batch.segs:  tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])
batch.clss:  tensor([[  0,  18,  36,  53,  84, 127, 166, 189, 252, 276, 327, 374, 436, 487,
           0,   0,   0,   0,   0],
        [  0,  15,  38,  71,  83, 129, 178, 232, 277, 289, 333, 374, 397, 411,
         421, 438, 455, 469, 492],
        [  0,  14,  37,  54,  89, 144, 168, 199, 223, 251, 267, 293, 333, 351,
         381, 399, 425,   0,   0],
        [  0,  51,  90, 109, 142, 175, 218, 245, 271, 281, 301, 331, 383, 424,
         452, 479,   0,   0,   0],
        [  0,  43,  77, 139, 173, 199, 216, 236, 258, 285, 297, 315, 327, 369,
         401, 430, 450, 500,   0]])
batch.mask:  tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]

Iteration:   0%|          | 1/17100 [00:06<30:51:56,  6.50s/it]

step:  1
batch:  <utils_nlp.models.transformers.bertsum.data_loader.Batch object at 0x7f8535ccc940>
batch.segs:  tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0]])
batch.clss:  tensor([[  0,  13,  28,  51,  97, 114, 141, 169, 215, 241, 266, 297, 335, 365,
         391, 461, 471, 499],
        [  0,  44,  89, 118, 144, 162, 173, 197, 209, 225, 242, 264, 283, 298,
         316, 344, 363, 391],
        [  0,  57,  91, 125, 157, 186, 208, 243, 260, 281, 306, 335, 348, 382,
         405, 428, 470, 483],
        [  0,   9,  32,  47,  82, 101, 118, 150, 194, 228, 241, 251, 293, 320,
         351, 442, 460, 501],
        [  0,  24,  69, 123, 139, 178, 192, 207, 222, 278, 288,   0,   0,   0,
           0,   0,   0,   0]])
batch.mask:  tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, T

Iteration:   0%|          | 2/17100 [00:11<29:24:36,  6.19s/it]

...

Iteration:   0%|          | 47/17100 [04:12<24:59:46,  5.28s/it]

step:  47
batch:  <utils_nlp.models.transformers.bertsum.data_loader.Batch object at 0x7f8535cccf98>
batch.segs:  tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 1, 1]])
batch.clss:  tensor([[  0,  24,  44,  77, 106, 143, 168, 196, 231, 269, 330, 360, 416, 437,
           0,   0,   0,   0,   0],
        [  0,  10,  29,  86, 113, 151, 192, 216, 228, 244, 266, 291, 311, 340,
         379, 411, 436, 477,   0],
        [  0,  69, 107, 131, 197, 229, 247, 263, 283, 330, 342, 358, 422, 442,
         464, 508,   0,   0,   0],
        [  0,  27,  41,  75, 103, 123, 162, 194, 217, 242, 263, 289, 323, 353,
         396, 420, 442, 468, 497],
        [  0,  39,  79,  96, 128, 144, 191, 203, 234, 263,   0,   0,   0,   0,
           0,   0,   0,   0,   0]])
batch.mask:  tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True,

Iteration:   0%|          | 48/17100 [04:18<25:12:54,  5.32s/it]

step:  48
batch:  <utils_nlp.models.transformers.bertsum.data_loader.Batch object at 0x7f8535cd9cf8>
batch.segs:  tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])
batch.clss:  tensor([[  0,  13,  36,  79, 123, 142, 157, 185, 200, 218, 248, 305, 321, 343,
         374, 384, 419, 436,   0,   0,   0,   0],
        [  0,  26,  34,  44,  57,  87, 134, 168, 186, 195, 228, 243, 264, 294,
         314, 358, 386, 425, 441, 472, 491, 510],
        [  0,  30,  44,  56,  72, 103, 130, 139, 159, 177, 194, 222, 258, 282,
         297, 329, 347, 360, 383, 410, 459, 501],
        [  0,  32,  57, 141, 165, 203, 262, 297, 358, 403, 441, 479,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0],
        [  0,  36,  55,  73, 111, 126, 138, 179, 209, 227, 253, 292,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0]])
batch.mask:  tensor([[True, True, True,  ..., True, True

KeyboardInterrupt: 

In [ ]:
summarizer.save_model(
    os.path.join(
        CACHE_DIR,
        "extsum_modelname_{0}_usepreprocess{1}_steps_{2}.pt".format(
            MODEL_NAME, USE_PREPROCSSED_DATA, MAX_STEPS
        ),
    )
)

### Model Evaluation

[ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)), or Recall-Oriented Understudy for Gisting Evaluation has been commonly used for evaluating text summarization.

In [ ]:
# for loading a previous saved model

model_filename = "dist_extsum_model.pt"
model_filepath = "/home/ubuntu/mnt/train/distilbert-base-german-cased/2007142250/"
model_path = os.path.join(model_filepath, model_filename)
summarizer = ExtractiveSummarizer(processor, MODEL_NAME, ENCODER, MAX_POS_LENGTH, CACHE_DIR)
summarizer.model.load_state_dict(torch.load(model_path, map_location="cpu"))

In [ ]:
if "segs" in ext_sum_test[0]: # preprocessed_data
    source = [i['src_txt'] for i in ext_sum_test]
    target = ["\n".join(i['tgt_txt'].split("<q>")) for i in ext_sum_test]
else:
    source = []
    temp_target = []
    for i in ext_sum_test:
        source.append(i["src_txt"]) 
        temp_target.append(" ".join(j) for j in i['tgt']) 
    target = [''.join(i) for i in list(temp_target)]

In [ ]:
%%time
sentence_separator = "\n"
prediction = summarizer.predict(ext_sum_test, num_gpus=NUM_GPUS, batch_size=BATCH_SIZE, sentence_separator=sentence_separator)

In [ ]:
rouge_scores = compute_rouge_python(cand=prediction, ref=target)
pprint.pprint(rouge_scores)

In [ ]:
prediction[0].replace("\n", " ")

In [ ]:
with open('sample_results.txt','w') as f:
    for i in range(len(prediction)):
        source_output = " ".join(source[i]) 
        f.write("Source Text: \n")
        f.write("\"" + source_output + "\" \n")
        f.write("\n")
        f.write("Source target: \n")
        f.write("\"" + target[i] + "\" \n")
        f.write("\n")
        f.write("Model Prediction: \n")
        f.write("\"" + prediction[i].replace("\n", " ") + "\" \n")        
        f.write("\n")
        f.write("======================================")        
        f.write("\n \n")

In [ ]:
target[10]

In [ ]:
target[2]

In [1]:
prediction[10]

NameError: name 'prediction' is not defined

In [ ]:
source

In [ ]:
# for testing
sb.glue("rouge_2_f_score", rouge_scores['rouge-2']['f'])

## Prediction on a single input sample

In [ ]:
source = """
Italien erlaubt nach tagelangem Zögern den etwa 180 Migranten auf dem privaten Rettungsschiff "Ocean Viking" den Wechsel auf das italienische Quarantäne-Schiff "Moby Zaza". Die Übernahme der aus Seenot geretteten Menschen sei für Montag geplant, hieß es am Samstagabend aus Quellen im Innenministerium in Rom. Zuvor hatte sich die Lage auf dem Schiff der Organisation SOS Méditerranée, das sich in internationalen Gewässern vor Sizilien befindet, zugespitzt.
Die Betreiber berichteten demnach von einem Hungerstreik unter den Geflüchteten. Verena Papke, Geschäftsführerin von SOS Méditerranée für Deutschland, hatte am Freitag von mehreren Suizidversuchen gesprochen. Die "Ocean Viking" hatte zudem den Notstand an Bord ausgerufen. Bis dahin waren mehrere Bitten um Zuweisung eines sicheren Hafens in Malta und Italien erfolglos geblieben.

Corona-Abstriche bei den Migranten geplant
Die Crew sandte die dringende Anfrage an die Behörden beider Länder zur Aufnahme von rund 45 Menschen, die in schlechter Verfassung seien. Italien schickte daraufhin am Samstag einen Psychiater und einen kulturellen Mediator aus Pozzallo für mehrere Stunden an Bord, berichteten beide Seiten. Danach kam die Erlaubnis aus Rom zur Übernahme auf die "Moby Zaza". Die Lage an Bord habe sich jedoch etwas entspannt, hieß es aus der italienischen Hauptstadt. Am Sonntag seien zunächst Corona-Abstriche bei den Migranten geplant.

Wie SOS Méditerranée am Samstag schrieb, nahm das Schiff in insgesamt vier Einsätzen am 25. und am 30. Juni etwa 180 Menschen aus dem Mittelmeer an Bord. Italien und Malta hatten sich in der Corona-Pandemie zu nicht sicheren Häfen erklärt. Trotzdem brechen Migranten von Libyen und Tunesien in Richtung Europa auf. Rom und Valletta nahmen zuletzt zwar wieder Menschen von privaten Schiffen auf, doch die Länder zögern mit der Zuweisung von Häfen oft lange. Sie fordern von anderen EU-Staaten regelmäßig Zusagen über die Weiterverteilung der Menschen."""

In [ ]:
test_dataset = SummarizationDataset(
    None,
    source=[source],
    source_preprocessing=[tokenize.sent_tokenize],
    word_tokenize=nltk.word_tokenize,
    language='german'
)
processor = ExtSumProcessor(model_name=MODEL_NAME,  cache_dir=CACHE_DIR)
preprocessed_dataset = processor.preprocess(test_dataset)

In [ ]:
preprocessed_dataset[0].keys()

In [ ]:
prediction = summarizer.predict(preprocessed_dataset, num_gpus=0, batch_size=1, sentence_separator="\n")

In [ ]:
prediction

## Clean up temporary folders

In [ ]:
if os.path.exists(DATA_PATH):
    shutil.rmtree(DATA_PATH, ignore_errors=True)
if os.path.exists(CACHE_DIR):
    shutil.rmtree(CACHE_DIR, ignore_errors=True)
if USE_PREPROCSSED_DATA:
    if os.path.exists(PROCESSED_DATA_PATH):
        shutil.rmtree(PROCESSED_DATA_PATH, ignore_errors=True)